## converting data from xarray to pandas format

In [2]:
import numpy as np
import pandas as pn
import xarray as xr
import matplotlib.pyplot as plt
import datetime as dt

### Saildrone data in xarray form

In [3]:
url = 'https://podaac-opendap.jpl.nasa.gov/opendap/hyrax/allData/insitu/L2/saildrone/Baja/saildrone-gen_4-baja_2018-sd1002-20180411T180000-20180611T055959-1_minutes-v1.nc'
ds = xr.open_dataset(url, drop_variables = {'WING_ANGLE','BARO_PRES_STDDEV', 'ROLL', 'PITCH', 'TEMP_AIR_STDDEV', 'RH_STDDEV', 'UWND_STDDEV', 'VWND_STDDEV', 'GUST_WND_STDDEV', 'TEMP_CTD_STDDEV','COND_MEAN', 'COND_STDDEV', 'SAL_MEAN', 'SAL_STDDEV', 'O2_CONC_UNCOR_MEAN', 'O2_CONC_UNCOR_STDDEV', 'O2_SAT_MEAN', 'O2_SAT_STDDEV', 'TEMP_O2_MEAN', 'TEMP_O2_STDDEV', 'CHLOR_MEAN', 'CHLOR_STDDEV', 'BKSCT_RED_MEAN', 'BKSCT_RED_STDDEV', 'CDOM_STDDEV', ' WWND_STDDEV', 'TEMP_IR_UNCOR_STDDEV' })
ds

<xarray.Dataset>
Dimensions:             (obs: 86839, trajectory: 1)
Coordinates:
  * trajectory          (trajectory) float32 1002.0
    time                (trajectory, obs) datetime64[ns] ...
    latitude            (trajectory, obs) float64 ...
    longitude           (trajectory, obs) float64 ...
Dimensions without coordinates: obs
Data variables:
    SOG                 (trajectory, obs) float64 ...
    COG                 (trajectory, obs) float64 ...
    HDG                 (trajectory, obs) float64 ...
    HDG_WING            (trajectory, obs) float64 ...
    BARO_PRES_MEAN      (trajectory, obs) float64 ...
    TEMP_AIR_MEAN       (trajectory, obs) float64 ...
    RH_MEAN             (trajectory, obs) float64 ...
    TEMP_IR_UNCOR_MEAN  (trajectory, obs) float64 ...
    UWND_MEAN           (trajectory, obs) float64 ...
    VWND_MEAN           (trajectory, obs) float64 ...
    WWND_MEAN           (trajectory, obs) float64 ...
    WWND_STDDEV         (trajectory, obs) float64 .

In [4]:
# swap obs for time
ds=ds.isel(trajectory=0)
ds = ds.swap_dims({'obs':'time'})
ds

<xarray.Dataset>
Dimensions:             (time: 86839)
Coordinates:
    trajectory          float32 1002.0
  * time                (time) datetime64[ns] 2018-04-11T18:00:00 ... 2018-06-11T05:57:00
    latitude            (time) float64 ...
    longitude           (time) float64 ...
Data variables:
    SOG                 (time) float64 ...
    COG                 (time) float64 ...
    HDG                 (time) float64 ...
    HDG_WING            (time) float64 ...
    BARO_PRES_MEAN      (time) float64 ...
    TEMP_AIR_MEAN       (time) float64 ...
    RH_MEAN             (time) float64 ...
    TEMP_IR_UNCOR_MEAN  (time) float64 ...
    UWND_MEAN           (time) float64 ...
    VWND_MEAN           (time) float64 ...
    WWND_MEAN           (time) float64 ...
    WWND_STDDEV         (time) float64 ...
    GUST_WND_MEAN       (time) float64 ...
    TEMP_CTD_MEAN       (time) float64 ...
    CDOM_MEAN           (time) float64 ...
Attributes:
    _NCProperties:             version=1|net

In [5]:
%time

CPU times: user 7 µs, sys: 2 µs, total: 9 µs
Wall time: 16.2 µs


In [ ]:
#resample to make time = 1 hr increments

pd_ds = ds.to_dataframe()
dshr = df.set_index('datetime').groupby(pd.Grouper(freq='1H')).mean()

#dshr=ds.resample(time='1h', skipna=True, label='left').mean()
dshr
# does data need to be averaged from beginning of hour to end, or from half way through one hour and half through the other??
#for now, we will average hours from beginning of hour to the end, how resample defaults
#data showed as 18:00 is data from the hour of 18:00 to 18:59

In [7]:
%time

CPU times: user 11 µs, sys: 11 µs, total: 22 µs
Wall time: 53.9 µs


# Do the math stuff in xarray
#then save as a dataframe, then to strings of the data 

In [8]:
time_shift = dshr.time + np.timedelta64(30,'m')

for i in range(dshr.size):
   # HR = str(int(100*(time_shift.dt.hour[i]+time_shift.dt.minute[i]/60)/100))
    #format string output and put hours in correct units (.01hr)
    HR = '{time_shift.dt.hour[i]+time_shift.dt.minute[i]/60:.2f}'
    HR = str(HR)
           
    time_str = str(time_shift.dt.year[i])+str(time_shift.dt.month)+str(time_shift.dt.day)+HR
print(time_str)

#format output to 2 decimal places
ds_hour = str(dshr['time.hour'])
precision = 2
ds_hour = '{ds_hour:.2f}'

# do the same for lat and lon 
lat_shift = ds.latitude
lon_shift = ds.longitude

# do lat and lon need shifted units or not?
for i in range(ds.size):
    LAT = '{lat_shift:.2f}'
    LON = '{lon_shift:.2f}'
    pos_str = str(LAT[i])+str(LON[i])
print(pos_str)

#ds_hour = dshr.resample(time = '.01H', skipna = True, label = 'left')
#put it in .01 hour for IMMA, need to do the same for LAT and LON

#ds_day = dshr['time.day']
#ds_day 

#ds_month = dshr['time.month']
#ds_month

#ds_year = dshr['time.year']
#ds_year

NameError: name 'dshr' is not defined

### Decode and store data as Pandas

In [ ]:
pd_ds = ds.to_dataframe()
pd_ds

In [ ]:
pd_ds[['time', 'PITCH']]

In [ ]:
#write dataframe to file
pd_ds.to_csv("test1.csv")  
#read it back
pd.read_csv("test1.csv").head()

### Convert new csv file to a text file

In [ ]:
# A simple program to create a formatted text file from a *.csv file.

csv_file = input('test1.csv')
txt_file = input('test1.txt')

try:
    my_input_file = open(csv_file, "r")
except IOError as e:
    print("I/O error({0}): {1}".format(e.errno, e.strerror))

if not my_input_file.closed:
    text_list = [];
    for line in my_input_file.readlines():
        line = line.split(",", 2)
        text_list.append(" ".join(line))
    my_input_file.close()

try:
    my_output_file = open(txt_file, "w")
except IOError as e:
    print("I/O error({0}): {1}".format(e.errno, e.strerror))

if not my_output_file.closed:
    my_output_file.write("#1\n")
    my_output_file.write("double({},{})\n".format(len(text_list), 2))
    for line in text_list:
        my_output_file.write("  " + line)
    print('File Successfully written.')
    my_output_file.close()